In [ ]:
import os

# Imports to support MIR tasks
import mirdata
import jams
import jams.display
import essentia.standard as ess
import librosa
import librosa.display

from torch.utils.data import Dataset
import pandas as pd
import torchaudio

In [ ]:
salami = mirdata.initialize("salami")

In [ ]:
salami.data_home = '/home/oriol_colome_font_epidemicsound_/Master-Thesis/datasets/SALAMI'

In [ ]:
salami_ids = salami.track_ids  # Return track IDs
salami_data = salami.load_tracks()  # Load Load all tracks in SALAMI

print(type(salami_data[salami_ids[0]])) # Print class
example_track = salami_data[salami_ids[0]]  # Visualize a single track
print(example_track)

In [ ]:
example_track_audio_path = salami_data[salami_ids[0]].audio_path #track's audio
print(f'Path to example track ---> {example_track_audio_path}')

In [ ]:
audio_dir = '/home/oriol_colome_font_epidemicsound_/Master-Thesis/datasets/SALAMI/audio'
ann_dir = '/home/oriol_colome_font_epidemicsound_/Master-Thesis/datasets/SALAMI/salami-data-public-hierarchy-corrections/annotations'

list_audios = [os.path.join(audio_dir, audio) for audio in os.listdir(audio_dir) if os.path.isfile(os.path.join(audio_dir, audio))]
list_ann = [os.path.join(ann_dir, ann) for ann in os.listdir(ann_dir) if os.path.isfile(os.path.join(ann_dir, ann))]

print(f'List of audio {list_audios}')
print(f'List of annotations {list_ann}')

In [ ]:
def sort_paths_by_last_folder(paths):
    # Split the paths into separate components (directories)
    split_paths = [path.split('/') for path in paths]

    # Extract the last component (directory)
    last_folders = [split_path[-1] for split_path in split_paths]

    # Sort the list based on the extracted last component
    sorted_paths = [path for _, path in sorted(zip(last_folders, paths))]

    return sorted_paths

In [ ]:
sorted_audios = sort_paths_by_last_folder(list_audios)
sorted_ann = sort_paths_by_last_folder(list_ann)

print(sorted_audios)
print(sorted_ann)

In [ ]:
for file1, file2 in zip(sorted_audios, sorted_ann):
    print(file1, file2)

In [ ]:
for index, track in enumerate(salami_data):
  print(f'Iteration --> {index}', 
        f'Track data --> {salami_data[salami_ids[int(index)]]}', 
        f'JAMS annotaion --> {salami_data[salami_ids[int(index)]].to_jams()}')

In [ ]:
#Create df here

In [ ]:
class SalamiDatasetLoader(Dataset):

    def __init__(self, annotations_file, audio_dir):
        self.annotations = pd.read_csv(annotations_file)
        self.audio_dir = audio_dir

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        audio_sample_path = self._get_audio_sample_path(index)
        label = self._get_audio_sample_label(index)
        signal, sr = torchaudio.load(audio_sample_path)
        return signal, label

    def _get_audio_sample_path(self, index):
        fold = f"fold{self.annotations.iloc[index, 5]}"
        path = os.path.join(self.audio_dir, fold, self.annotations.iloc[
            index, 0])
        return path

In [ ]:
#ANNOTATIONS_FILE = "XXXX.csv"
AUDIO_DIR = '/content/drive/MyDrive/SALAMI/audio'

In [ ]:
#SDL = SalamiDatasetLoader(ANNOTATIONS_FILE, AUDIO_DIR)

In [ ]:
#print(f"There are {len(SDL)} samples in the dataset.")

In [ ]:
#signal, label = SDL[0]